In [1]:
import os

import requests
import json
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [2]:
index_name = "conversations"
index_path = "../indexes/conversations"
embeddings = OpenAIEmbeddings()
# db = FAISS.from_texts(texts=['9'], embedding=embeddings)
# db.save_local(folder_path=index_path, index_name=index_name) 
# db = FAISS.load_local(folder_path=index_path, index_name=index_name, embeddings=embeddings)


In [3]:
def gpt_system_user(
    system_message: str, user_message: str, model: str = "gpt-3.5-turbo"
):
    """Para usar en notebooks"""
    # Hyper 2
    # POST https://yoizenia.openai.azure.com/openai/deployments/GPT35Turbo/chat/completions

    try:
        response = requests.post(
            url="https://api.openai.com/v1/chat/completions",
            params={
                "temperature": "0",
            },
            headers={
                "Content-Type": "application/json",
                "Authorization": f"Bearer {OPENAI_API_KEY}",
            },
            data=json.dumps(
                {
                    "model": model,
                    "messages": [
                        {"content": system_message, "role": "system"},
                        {"content": user_message, "role": "user"},
                    ]
                }
            ),
            timeout=10,
        )
        choices = response.json()["choices"]
        answer = {
            "status_code": response.status_code,
            "text": choices[0]["message"],
        }
        return answer

    except requests.exceptions.RequestException:
        print("HTTP Request failed")
        return None


In [12]:
condensed_contexts =  [
        {
          "messages": [
              {
                "role": "user",
                "content": "Voy a sacar un pasaje para la familia de buenos aires a cordoba"
              },
              {
                "role": "assistant",
                "content": "quienes son los pasajeros?"
              },
              {
                "role": "user",
                "content": "Ana es mi hija mayor, tiene 20 años. Juan es mi hijo menor, tiene 10 años. Mi esposa se llama Maria y tiene 40 años. Yo me llamo Pedro y tengo 45 años"
              },
              {
                "role": "assistant",
                "content": "en que fecha quiere viajar?"
              },
              {
                "role": "user",
                "content": "del 20 de agosto al 30 de agosto"
              },
              {
                  "role": "assistant",
                  "content": "perfecto, salen el 20 de agosto a las 10:00 y vuelven el 30 de agosto a las 18:00. El precio total es de 1000 dolares. Desea confirmar la compra?"
              },
              {
                  "role": "user",
                  "content": "no, para, puedo pagar con mis millas?"
              },
              {
                  "role": "assistant",
                  "content": "Usted cuenta con 200 millas. Desea usarlas?"
              },
              {
                "role": "user",
                "content": "si"
              },
              {
                "role": "assistant",
                "content": "perfecto, el precio total es de 800 dolares usando sus millas. Desea confirmar la compra?"
              },
              {
                "role": "user",
                "content": "si"
              },
              {
                "role": "assistant",
                "content": "Gracias por su compra. El numero de su reserva es 123456"
              }
           ],
          "metadata": {
            "session_id": "1",
            "person_id": "20",
            "context_id": "1",
            "created_at": "20200825T12:00:00.000Z",
          },
        },
        {
            "messages": [
                {
                "role": "user",
                "content": "Quiero subir de categoria en el programa de viajero frecuente"
                },
                {
                    "role": "assistant",
                    "content": "que categoria tiene actualmente?"
                },
                { 
                  "role": "user",
                  "content": "soy categoria 1"
                },
                {
                  "role": "assistant",
                  "content": "para subir de categoria debe tener 1000 millas. Cuantas millas tiene actualmente?"
                },
                {
                  "role": "user",
                  "content": "tengo 500 millas"
                },
                {
                  "role": "assistant",
                  "content": "puede comprar 500 millas por 100 dolares. Desea comprarlas?"
                },
                {
                  "role": "user",
                  "content": "si"
                },
                {
                  "role": "assistant",
                  "content": "Gracias por su compra. Ahora tiene 1000 millas y es categoria 2"
                }
            ],
            "metadata": {
                "session_id": "1",
                "person_id": "20",
                "context_id": "2",
                "created_at": "20220825T18:00:00.000Z",                
              }
        }
  ]


In [17]:
db = FAISS.from_texts(texts=['notebook-14'], embedding=embeddings)
for context in condensed_contexts:
        # Join all messages in one string by a comma
        messages = ''.join([str(message) for message in context['messages']])
        # print(messages)
        # print(context['metadata'])
        extraction = gpt_system_user(
            system_message="""
              Extract every meaningful information about the user from the provided conversation.
              Ask yourself, what happened in the conversation?
              What data about the user did you learn?
              At what date and time was this data gathered?

              Create a summary of everything you learned about the user.

              Use this format:

              On [date] at [time] the [user's / user's related person] [action] [data  learned about the user].
              """,
            user_message=f" METADATA: ```{context['metadata']} CONVERSATION: ```{messages}``` ```",
            model="gpt-3.5-turbo",
        )
        if extraction['status_code'] != 200:
            print(f"Error: Openai API returned status code {extraction['status_code']}")
            continue
        else:
            print(f"{extraction['text']['content']} \n\n\n///////////////////////////\n\n\n")
            db.add_texts(
                texts=[extraction['text']['content']], 
                metadatas=[context['metadata']],
            )

On August 25th at 12:00 PM, the user provided the following information:
- The user's name is Pedro.
- Pedro has a wife named Maria who is 40 years old.
- Pedro has a daughter named Ana who is 20 years old.
- Pedro has a son named Juan who is 10 years old.

In the conversation, the user mentioned that they wanted to purchase tickets for their family from Buenos Aires to Cordoba. They provided the dates of travel as August 20th to August 30th. The assistant confirmed the details and offered a total price of $1000. The user then asked about using their miles for payment, to which the assistant informed them that they have 200 miles available. The user confirmed the use of miles and the assistant adjusted the total price to $800. The user then confirmed the purchase. The conversation ended with the assistant providing the user with a reservation number of 123456. 


///////////////////////////



On 2022-08-25 at 18:00:00, the user expressed their desire to upgrade their frequent traveler

Mezclando data y fechas de dos contextos

In [18]:
user_input = "cuantas millas tenia en el momento de hacer el viaje a cordoba?"
filter = {"person_id": "20"}
db_results = db.similarity_search_with_score(query=user_input, embeddings=embeddings, filter=filter, k=2)
context_k2 = [f'\n {db_results[i][0].page_content} {db_results[i][0].metadata}\n---------------\n\n' for i in range(len(db_results))]
context = context_k2
for c in context:
    print(c)
system_prompt = f"""
You are the best personal assistant of SAIA Air.
Your job is to help users in their travel needs.
Use the user's name in your answers.
You will find past conversations between you and the user between backticks.
Use them to answer the user's question.
To reason about dates, have in mind that today is September 13th, 2023.
Wait before answering, check when the data was gathered first.
Take a deep breath and work on this problem step-by-step.



```{context}```

"""
# answer = gpt_system_user(system_message=system_prompt, user_message=user_input, model="gpt-4")
answer = gpt_system_user(system_message=system_prompt, user_message=user_input)
# answer["text"]["content"]



 On August 25th at 12:00 PM, the user provided the following information:
- The user's name is Pedro.
- Pedro has a wife named Maria who is 40 years old.
- Pedro has a daughter named Ana who is 20 years old.
- Pedro has a son named Juan who is 10 years old.

In the conversation, the user mentioned that they wanted to purchase tickets for their family from Buenos Aires to Cordoba. They provided the dates of travel as August 20th to August 30th. The assistant confirmed the details and offered a total price of $1000. The user then asked about using their miles for payment, to which the assistant informed them that they have 200 miles available. The user confirmed the use of miles and the assistant adjusted the total price to $800. The user then confirmed the purchase. The conversation ended with the assistant providing the user with a reservation number of 123456. {'session_id': '1', 'person_id': '20', 'context_id': '1', 'created_at': '20200825T12:00:00.000Z'}
---------------



 On 2022

In [19]:
answer["text"]["content"]

'Según la información recopilada en las conversaciones anteriores, en el momento de hacer el viaje a Córdoba, Pedro tenía 200 millas disponibles.'

Aca hay un error de razonamiento pero es mucho pedir para un prompt tan corto.

Posibles pasos siguientes:

- Navegar el grafo de relaciones del usuario y cambiar el punto focal actual. "este turno es para mi madre"
- Usar el mismo metodo para guardar cosas como "La madre del usuario tiene un turno para el dentista el 1 de enero de 2021"
- Sumar muchos mas datos personales y ver si hay confusiones
- Simular llamados a APIs desde diferentes relaciones del usuario y ver si hay confusiones
